---
title: "Poisson Regression Examples"
author: "Jaqueline Vallejo Hinojosa"
date: today
callout-appearance: minimal # this hides the blue "i" icon on .callout-notes
---





## Blueprinty Case Study

### Introduction

Blueprinty is a small firm that makes software for developing blueprints specifically for submitting patent applications to the US patent office. Their marketing team would like to make the claim that patent applicants using Blueprinty's software are more successful in getting their patent applications approved. Ideal data to study such an effect might include the success rate of patent applications before using Blueprinty's software and after using it. Unfortunately, such data is not available. 

However, Blueprinty has collected data on 1,500 mature (non-startup) engineering firms. The data include each firm's number of patents awarded over the last 5 years, regional location, age since incorporation, and whether or not the firm uses Blueprinty's software. The marketing team would like to use this data to make the claim that firms using Blueprinty's software are more successful in getting their patent applications approved.


### Data


In [ ]:
import pandas as pd 

blueprinty = pd.read_csv('blueprinty.csv')
blueprinty.head()

patents_num = blueprinty.groupby('iscustomer')['patents'].mean()
patents_num

import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(data=blueprinty, x='patents', hue='iscustomer', multiple='stack', bins=20)
plt.title('Histogram of Number of Patents by Customer Status')
plt.xlabel('Number of Patents')
plt.ylabel('Count')
plt.show()

* The histogram shows that non-customers consistently outnumber customers across patent counts, with most entities, regardless of customer status, tend to hold 3–4 patents.

Blueprinty customers are not selected at random. It may be important to account for systematic differences in the age and regional location of customers vs non-customers.


In [ ]:
region_counts = blueprinty.groupby(['iscustomer', 'region']).size().unstack()
#print(region_counts)

region_counts.T.plot(kind='bar', stacked=True)
plt.title('Customer Status by Region')
plt.xlabel('Region')
plt.ylabel('Count')
plt.legend(title='Customer Status')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

* The chart shows that the Northeast has the highest overall customer count and a relatively balanced customer-to-non-customer ratio, while all other regions are dominated by non-customers. 


In [ ]:
import matplotlib.pyplot as plt

blueprinty.boxplot(column='age', by='iscustomer')
plt.title('Age Distribution by Customer Status')
plt.suptitle('')
plt.xlabel('Customer Status')
plt.ylabel('Age')
plt.show()

* The age distribution for customers and non-customers is similar, with both groups centered around the mid-to-late 20s, though customers show slightly higher age variability and a marginally higher median


### Estimation of Simple Poisson Model

Since our outcome variable of interest can only be small integer values per a set unit of time, we can use a Poisson density to model the number of patents awarded to each engineering firm over the last 5 years. We start by estimating a simple Poisson model via Maximum Likelihood.


For a random variable $Y \sim \text{Poisson}(\lambda)$, we consider the probability mass function (PMF):

$$
f(Y \mid \lambda) = \frac{e^{-\lambda} \lambda^Y}{Y!}
$$

If we observe **independent data points** $Y_1, Y_2, \dots, Y_n$ from a Poisson distribution with the same rate parameter $\lambda$, then the **likelihood function** is the product of their individual probabilities:

$$
\mathcal{L}(\lambda; Y_1, \dots, Y_n) = \prod_{i=1}^n \frac{e^{-\lambda} \lambda^{Y_i}}{Y_i!}
$$

$$
= e^{-n\lambda} \lambda^{\sum_{i=1}^n Y_i} \prod_{i=1}^n \frac{1}{Y_i!}
$$


In [ ]:
import numpy as np
from scipy.special import gammaln

def poisson_loglikelihood(lam, Y):
 
    Y = np.array(Y)
    
    if lam <= 0:
        return -np.inf  # log-likelihood undefined for non-positive lambda

    loglik = np.sum(-lam + Y * np.log(lam) - gammaln(Y + 1))
    return loglik

In [ ]:
Y = blueprinty['patents'].values 

lambda_vals = np.linspace(0.1, 10, 200)

loglik_vals = [poisson_loglikelihood(lam, Y) for lam in lambda_vals]

plt.figure(figsize=(8, 5))
plt.plot(lambda_vals, loglik_vals, label='Log-Likelihood')
plt.xlabel('Lambda (λ)')
plt.ylabel('Log-Likelihood')
plt.title('Poisson Log-Likelihood as a Function of λ')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
lambda_mle = Y.mean()
print("MLE of lambda (λ̂):", lambda_mle)

In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.special import gammaln

def neg_poisson_loglikelihood(lam):
    if lam <= 0:
        return np.inf  
    loglik = np.sum(-lam + Y * np.log(lam) - gammaln(Y + 1))
    return -loglik  
initial_lambda = np.array([1.0])

result = minimize(neg_poisson_loglikelihood, initial_lambda, bounds=[(1e-6, None)])

lambda_mle = result.x[0]

print("MLE of lambda (λ̂) from optimization:", lambda_mle)

### Estimation of Poisson Regression Model

Next, we extend our simple Poisson model to a Poisson Regression Model such that $Y_i = \text{Poisson}(\lambda_i)$ where $\lambda_i = \exp(X_i'\beta)$. The interpretation is that the success rate of patent awards is not constant across all firms ($\lambda$) but rather is a function of firm characteristics $X_i$. Specifically, we will use the covariates age, age squared, region, and whether the firm is a customer of Blueprinty.


In [ ]:
import numpy as np
from scipy.special import gammaln

def poisson_regression_loglikelihood(beta, Y, X):
    eta = X @ beta
    lam = np.exp(eta)
    loglik = np.sum(-lam + Y * eta - gammaln(Y + 1))
    return -loglik

In [ ]:
import numpy as np
import pandas as pd

# Standardize age and create age_squared
blueprinty['age'] = (blueprinty['age'] - blueprinty['age'].mean()) / blueprinty['age'].std()
blueprinty['age_squared'] = blueprinty['age'] ** 2

# One-hot encode region (excluding one as reference group)
X_region = pd.get_dummies(blueprinty['region'], drop_first=True)

X = pd.concat([
    pd.Series(1, index=blueprinty.index, name='intercept'),
    blueprinty[['age', 'age_squared']],
    X_region,
    blueprinty['iscustomer']
], axis=1).astype(float)

column_names = X.columns.tolist()
X = X.values

Y = blueprinty['patents'].values.astype(float)

from scipy.optimize import minimize

initial_beta = np.zeros(X.shape[1])

result = minimize(
    poisson_regression_loglikelihood,
    initial_beta,
    args=(Y, X),
    method='BFGS'
)

beta_mle = result.x
hessian_inv = result.hess_inv  # inverse of Hessian (variance-covariance matrix)

se_beta = np.sqrt(np.diag(hessian_inv))

results_df = pd.DataFrame({
    'Coefficient': beta_mle,
    'Std. Error': se_beta
}, index=column_names)

print(results_df)

In [ ]:
import statsmodels.api as sm

model = sm.GLM(Y, X, family=sm.families.Poisson())
results = model.fit()
print(results.summary())

glm_coef = results.params
glm_se = results.bse

comparison_df = pd.DataFrame({
    'Custom Coef': beta_mle,
    'GLM Coef': glm_coef,
    'Custom SE': se_beta,
    'GLM SE': glm_se
}, index=column_names)

print(comparison_df)

## Results
The Poisson regression model indicates that age has a significant negative effect on patent counts, with each standard deviation increase in age associated with a 5.6% decrease in expected patents. The negative and significant age-squared term suggests a concave relationship, indicating that patent activity peaks and then declines with age. Customers are estimated to file 23% more patents than non-customers, a statistically significant difference. Regional differences are small and not statistically significant, suggesting that location has little effect on patent counts after accounting for other variables.


In [ ]:
X_0 = X.copy()
X_1 = X.copy()

# Assume 'iscustomer' is the **last column** in X
X_0[:, -1] = 0  
X_1[:, -1] = 1  

eta_0 = X_0 @ beta_mle
eta_1 = X_1 @ beta_mle

y_pred_0 = np.exp(eta_0)  
y_pred_1 = np.exp(eta_1)  

patent_diff = y_pred_1 - y_pred_0
average_diff = np.mean(patent_diff)
print("Average increase in predicted patents due to Blueprinty's software:", average_diff)

Based on the Poisson regression model, Blueprinty’s software is associated with an average increase of approximately 0.79 patents per firm. This estimate represents the expected difference in patent output if every firm were a customer versus if none were, holding age and region constant. The result suggests that Blueprinty's software has a positive and meaningful effect on patent generation, supporting its potential value as an innovation-enhancing tool.


## AirBnB Case Study

### Introduction

AirBnB is a popular platform for booking short-term rentals. In March 2017, students Annika Awad, Evan Lebo, and Anna Linden scraped of 40,000 Airbnb listings from New York City.  The data include the following variables:

:::: {.callout-note collapse="true"}
### Variable Definitions

    - `id` = unique ID number for each unit
    - `last_scraped` = date when information scraped
    - `host_since` = date when host first listed the unit on Airbnb
    - `days` = `last_scraped` - `host_since` = number of days the unit has been listed
    - `room_type` = Entire home/apt., Private room, or Shared room
    - `bathrooms` = number of bathrooms
    - `bedrooms` = number of bedrooms
    - `price` = price per night (dollars)
    - `number_of_reviews` = number of reviews for the unit on Airbnb
    - `review_scores_cleanliness` = a cleanliness score from reviews (1-10)
    - `review_scores_location` = a "quality of location" score from reviews (1-10)
    - `review_scores_value` = a "quality of value" score from reviews (1-10)
    - `instant_bookable` = "t" if instantly bookable, "f" if not

::::


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("airbnb.csv")  
print(df.columns)

df.columns = df.columns.str.strip()

relevant_cols = [
    'room_type', 'bathrooms', 'bedrooms', 'price', 'number_of_reviews',
    'review_scores_cleanliness', 'review_scores_location', 'review_scores_value',
    'instant_bookable'
]

df_clean = df[relevant_cols].copy()

df_clean = df_clean.dropna()

df_clean['price'] = df_clean['price'].replace('[\$,]', '', regex=True).astype(float)

# Convert instant_bookable to binary
df_clean['instant_bookable'] = df_clean['instant_bookable'].map({'t': 1, 'f': 0})

df_clean = pd.get_dummies(df_clean, columns=['room_type'], drop_first=True)

# Define y and X
y = df_clean['number_of_reviews'].astype(float)
X = df_clean.drop(columns=['number_of_reviews'])

# Add intercept and ensure numeric types
X = sm.add_constant(X).astype(float)

# Fit Poisson model
poisson_model = sm.GLM(y, X, family=sm.families.Poisson()).fit()

# Display results
print(poisson_model.summary())

The model shows that listings with **more bedrooms** receive **more reviews**, while those with **more bathrooms** receive fewer. Specifically, each extra bedroom is linked to about an **7% increase** in reviews, while each extra bathroom is linked to about a **12% decrease**. **Cleanliness ratings** have a strong positive effect: higher cleanliness scores lead to significantly more reviews. In contrast, higher **location** and **value** scores are linked to **fewer reviews**. Listings with **instant booking** enabled get **about 33% more reviews**, suggesting guests prefer the convenience. Compared to entire homes, **shared rooms** get **about 25% fewer reviews**, while **private rooms** get slightly fewer reviews. These results show that cleanliness, convenience, and room type are key drivers of variation in review counts across listings.
